In [8]:
# Dependencies
import numpy as np
import pandas as pd
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

In [7]:
import sqlite3
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [10]:
engine = create_engine('sqlite:///db/newfinaldata.sqlite')
# Declare a Base using `automap_base()`
Base = automap_base()
# Use the Base class to reflect the database tables
Base.prepare(engine, reflect=True)

In [11]:
# Save references to each table
Movies = Base.classes.movie_data
Images = Base.classes.images
Upcoming = Base.classes.upcoming

In [13]:
# Create a session
session = Session(engine)

In [17]:
def clean_movies_two(movies, title):
    counter = 0
    for movie in movies:
        if movie == title:
            movies.remove(movie)
            counter += 1

    if counter == 0:
        del movies[-1]
    
    return movies

In [4]:
def recommend_upcoming(movie_name, genre):

    cnx = sqlite3.connect('db/newfinaldata.sqlite')
    df_upcoming = pd.read_sql_query("SELECT * FROM upcoming", cnx)

    # Break up the big genre string into a string array
    df_upcoming['genre'] = df_upcoming['genre'].str.split(',')
    
    print(df_upcoming)

    dict1 = {
        "name": movie_name,
        "genre": genre
    }
    df_upcoming.append(dict1, ignore_index=True)

    # Convert genres to string value
    df_upcoming['genre'] = df_upcoming['genre'].fillna("").astype('str')

    tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
    tfidf_matrix = tf.fit_transform(df_upcoming['genre'])
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

    # Build a 1-dimensional array with movie titles
    titles = df_upcoming['name']
    indices = pd.Series(df_upcoming.index, index=df_upcoming['name'])

    newtitle = movie_name
    idx = indices[newtitle]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[0:3]
    movie_indices = [i[0] for i in sim_scores]

    recommendations =  titles.iloc[movie_indices]

    return recommendations

In [56]:
def get_genre(movie):
    sel = [
        Movies.name,
        Movies.genre
    ]

    table = session.query(*sel).filter(Movies.name == movie).all()
    print(table)

    movie_data = []
    for results in table:
        movie = {}
        movie["Title"] = results[0]
        movie["Genre"] = results[1]
        movie_data.append(movie)
    
    movie_data = movie_data[0]
    
    # movie_data[1] = movie_data[1].str.split("|")
    print(movie_data)

    upcoming_movies = recommend_upcoming(movie, movie_data["Genre"])
    print(upcoming_movies)

    try:
        movie_recommendation = clean_movies_two(upcoming_movies, movie)
    except KeyError:
        movie_recommendation: []

    return movie_recommendation


In [57]:
movie = get_genre('Avatar')

[('Avatar', 'Action|Adventure|Fantasy|Sci-Fi')]
{'Title': 'Avatar', 'Genre': 'Action|Adventure|Fantasy|Sci-Fi'}


KeyError: "None of [Index(['Title', 'Genre'], dtype='object', name='name')] are in the [index]"

In [47]:
cnx = sqlite3.connect('db/newfinaldata.sqlite')
df_movie = pd.read_sql_query("SELECT name, genre FROM movie_data WHERE name = 'Avatar'", cnx)
df_movie = df_movie['genre'].str.split("|")
#df_movie = df_movie['genre'].fillna("").astype('str')
df_movie

0    [Action, Adventure, Fantasy, Sci-Fi]
Name: genre, dtype: object

In [35]:
upcoming['genre']

0                    [Action,  Comedy            ]
1      [Animation,  Adventure,  Drama            ]
2                     [Comedy,  Drama            ]
3        [Action,  Adventure,  Comedy            ]
4                             [Horror            ]
5                 [Adventure,  Family            ]
6             [Action,  Crime,  Drama            ]
7                     [Comedy,  Drama            ]
8              [Drama,  History,  War            ]
9     [Animation,  Adventure,  Comedy            ]
10          [Comedy,  Drama,  Mystery            ]
11       [Horror,  Mystery,  Thriller            ]
12    [Animation,  Adventure,  Comedy            ]
13                 [Horror,  Thriller            ]
14       [Adventure,  Drama,  Mystery            ]
15     [Action,  Adventure,  Thriller            ]
16                             [Drama            ]
17    [Animation,  Adventure,  Comedy            ]
18        [Action,  Horror,  Thriller            ]
19          [Action,  Horror,  